In [2]:
import pandas as pd
import numpy as np

In [3]:
# read in the data
df = pd.read_csv('data/ecommerce-data.csv')
df['ProductID'] = df['ProductName'] + df['ProductCategory'] + df['ProductSubCategory']
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                5000 non-null   object 
 1   ProductName         5000 non-null   object 
 2   ProductCategory     5000 non-null   object 
 3   ProductSubCategory  5000 non-null   object 
 4   ProductPrice        5000 non-null   object 
 5   CustomerName        5000 non-null   object 
 6   CustomerEmail       4493 non-null   object 
 7   CustomerAddress     5000 non-null   object 
 8   CustomerPhone       5000 non-null   object 
 9   CustomerSegment     5000 non-null   object 
 10  SupplierName        5000 non-null   object 
 11  SupplierLocation    5000 non-null   object 
 12  SupplierContact     4516 non-null   object 
 13  ShipperName         5000 non-null   object 
 14  ShippingMethod      5000 non-null   object 
 15  QuantitySold        5000 non-null   int64  
 16  TotalAmount

In [3]:
# Check for nulls
df.isnull().sum()

Date                    0
ProductName             0
ProductCategory         0
ProductSubCategory      0
ProductPrice            0
CustomerName            0
CustomerEmail         507
CustomerAddress         0
CustomerPhone           0
CustomerSegment         0
SupplierName            0
SupplierLocation        0
SupplierContact       484
ShipperName             0
ShippingMethod          0
QuantitySold            0
TotalAmount             0
DiscountAmount          0
NetAmount               0
StockReceived           0
StockSold               0
StockOnHand             0
dtype: int64

In [10]:
df.head()

,Date,ProductName,ProductCategory,ProductSubCategory,ProductPrice,CustomerName,CustomerEmail,CustomerAddress,CustomerPhone,CustomerSegment,...,SupplierContact,ShipperName,ShippingMethod,QuantitySold,TotalAmount,DiscountAmount,NetAmount,StockReceived,StockSold,StockOnHand
0,2023-09-14,Nathaniel,Electronics,Camera,0.01,Colleen Kelly,maryhurst@example.org,"354 Mcdowell Turnpike, Port Charles, CT 95318",908.610.2711x8507,Silver,...,6538306661,and Sons,Ground,49,31965.15,121.07,31844.08,475,127,348
1,2023-02-11,NonExistentProduct,Electronics,Mobile,847.43,Joel Wright,sandersvictoria@example.org,"24740 Fox Villages, New Tracie, MA 53038",+1-408-938-0389x952,Gold,...,NaN,PLC,Air,73,61862.39,91.09,61771.30,487,243,244
2,2021-11-12,Angela,InvalidCategory,Action Figures,386.57,Thomas Sawyer,ospence@example.net,"769 Joe Trail, East Terri, CA 43813",001-929-516-1919x39288,Gold,...,+1-588-750-7646,PLC,Sea,89,34404.73,10.56,34394.17,341,188,153
3,11-15-2021,Amy,Home & Garden,Decor,364.01,Tyler Gardner,christopherjohnson@example.com,"27783 Olivia Centers, Williamsmouth, AL 09809",8907712983,Gold,...,805-650-6257x5876,LLC,Air,3,1092.03,69.06,1022.97,500,124,376
4,2023-04-22,Nathaniel,Electronics,Camera,652.35,Meagan Peterson,epowell@example.net,"25357 Blackwell Locks, Andreabury, MH 27857",9999921886,Gold,...,6538306661,Ltd,Sea,75,48926.25,137.17,48789.08,429,351,78


## Date exploration

In [4]:
# Check for Date column formats
df['Date'].head(20)

0     2023-09-14
1     2023-02-11
2     2021-11-12
3     11-15-2021
4     2023-04-22
5     2022-12-08
6     2023-09-13
7     2023-07-19
8     2023-08-20
9     2023-07-15
10    09-25-2023
11    2022-05-06
12    2023-09-19
13    2022-10-19
14    2022-08-30
15    2023-01-25
16    2022-11-02
17    2023-07-29
18    2022-02-09
19    2023-04-25
Name: Date, dtype: object

In [5]:
df['Date'].str.contains('-').sum() 

5010

In [7]:
dfdate = df[df['Date'].str.contains('-')]

In [8]:
# List of possible date patterns
date_patterns = [
    r'^\d{4}-\d{2}-\d{2}$',  # yyyy-mm-dd or yyyy-dd-mm
    r'^\d{2}-\d{2}-\d{4}$',  # dd-mm-yyyy or mm-dd-yyyy
]

# Initialize a dictionary to store the counts for the two desired patterns
desired_pattern_counts = {
    'yyyy-mm-dd or yyyy-dd-mm': 0,
    'dd-mm-yyyy or mm-dd-yyyy': 0,
}

# Iterate through the patterns and count the matching elements for the desired patterns
for pattern in date_patterns:
    dfdate['ContainsPattern'] = dfdate['Date'].str.contains(pattern)
    count = dfdate['ContainsPattern'].sum()
    
    if pattern == r'^\d{4}-\d{2}-\d{2}$' :
        desired_pattern_counts['yyyy-mm-dd or yyyy-dd-mm'] += count
    elif pattern == r'^\d{2}-\d{2}-\d{4}$':
        desired_pattern_counts['dd-mm-yyyy or mm-dd-yyyy'] += count

# Print the counts for the two desired patterns
for pattern, count in desired_pattern_counts.items():
    print(f"Pattern '{pattern}': {count} elements")

# Optional: Drop the temporary 'ContainsPattern' column
dfdate.drop(columns=['ContainsPattern'], inplace=True)


Pattern 'yyyy-mm-dd or yyyy-dd-mm': 4512 elements
Pattern 'dd-mm-yyyy or mm-dd-yyyy': 498 elements


In [10]:
# split by - and assign each elemt into a list
datadf = pd.DataFrame()
datadf["elm1"] = df['Date'].str.split('-').str[0]
datadf["elm2"] = df['Date'].str.split('-').str[1]
datadf["elm3"] = df['Date'].str.split('-').str[2]

# get unique values for each column
print(datadf["elm1"].unique())
print(datadf["elm2"].unique())
print(datadf["elm3"].unique())

['2023' '2021' '11' '2022' '09' '10' '12' '05' '01' '03' '06' '02' '04'
 '07' '08']
['09' '02' '11' '15' '04' '12' '07' '08' '25' '05' '10' '01' '03' '27'
 '06' '16' '14' '29' '24' '26' '21' '30' '13' '23' '18' '31' '28' '19'
 '17' '22' '20']
['14' '11' '12' '2021' '22' '08' '13' '19' '20' '15' '2023' '06' '30' '25'
 '02' '29' '09' '17' '03' '05' '04' '28' '10' '16' '07' '31' '18' '2022'
 '21' '01' '27' '26' '23' '24']


In [19]:
cat1 = datadf[datadf["elm1"].str.contains('^\d{4}')]
cat2 = datadf[datadf["elm3"].str.contains('^\d{4}')]

print("yyyy-mm-dd \n")
print(cat1["elm2"].unique())
print("mm-dd-yyyy \n")
print(cat2["elm1"].unique())

yyyy-mm-dd 

['09' '02' '11' '04' '12' '07' '08' '05' '10' '01' '03' '06']
mm-dd-yyyy 

['11' '09' '10' '12' '05' '01' '03' '06' '02' '04' '07' '08']


## ProductName & ProductCategory & ProductSubCategory exploration

In [24]:
print(df["ProductName"].unique())
print(df["ProductCategory"].unique())
print(df["ProductSubCategory"].unique())

['Nathaniel' 'NonExistentProduct' 'Angela' 'Amy' 'Justin' 'Shannon'
 'Martha' 'Elizabeth' 'Jamie' 'Robert' 'Shawn' 'Anthony' 'Pamela' 'Fred'
 'Wanda' 'Cynthia' 'Marvin' 'Katie' 'Tammy' 'Michael' 'Kristine']
['Electronics' 'InvalidCategory' 'Home & Garden' 'Books' 'Clothing' 'Toys']
['Camera' 'Mobile' 'Action Figures' 'Decor' 'Headphones' 'Gardening Tools'
 'Fiction' 'Shirt' 'Academic' 'Educational' 'Non-fiction' 'Dolls'
 'Furniture']


## ProductPrice exploration

In [32]:
# replace the "InvalidPrice" with NaN
df["ProductPrice"].replace("InvalidPrice", np.nan, inplace=True)
df["ProductPrice"].astype(float).describe()

count     4561.00000
mean     11556.88205
std      28715.89313
min          0.01000
25%        364.01000
50%        652.35000
75%        930.30000
max      99999.99000
Name: ProductPrice, dtype: float64

## CustomerName exploration

In [36]:
len(df["CustomerName"].unique())

4846

## CustomerEmail & CustomerAddress exploration

In [19]:
df.drop_duplicates(keep='first', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                5000 non-null   object 
 1   ProductName         5000 non-null   object 
 2   ProductCategory     5000 non-null   object 
 3   ProductSubCategory  5000 non-null   object 
 4   ProductPrice        5000 non-null   object 
 5   CustomerName        5000 non-null   object 
 6   CustomerEmail       4517 non-null   object 
 7   CustomerAddress     5000 non-null   object 
 8   CustomerPhone       5000 non-null   object 
 9   CustomerSegment     5000 non-null   object 
 10  SupplierName        5000 non-null   object 
 11  SupplierLocation    5000 non-null   object 
 12  SupplierContact     4516 non-null   object 
 13  ShipperName         5000 non-null   object 
 14  ShippingMethod      5000 non-null   object 
 15  QuantitySold        5000 non-null   int64  
 16  TotalAmount

In [20]:
print(len(df["CustomerEmail"].unique()))
print(len(df["CustomerAddress"].unique()))

4451
5000


In [48]:
print(df['CustomerEmail'].str.contains(r'^.+@[^\.].*\.[a-z]{2,}$').value_counts())
print("Nulls count : " , df['CustomerEmail'].isnull().sum())

CustomerEmail
True    4503
Name: count, dtype: int64
Nulls count :  507


In [17]:
# for df["CustomerEmail"] column, check if the same CustomerName has an non-null email in another row
# if yes, replace the null with the non-null email
# if no, replace the null with the CustomerName
df["CustomerEmail"].fillna(df.groupby("CustomerName")["CustomerEmail"].transform('first'), inplace=True)
df["CustomerEmail"].isna().sum()


483

In [56]:
print("Redundent Name per customer's mails count : " , len(df[df.groupby('CustomerEmail')['CustomerName'].transform('count') > 1]))
print("Redundent Name per customer's adress count : " , len(df[df.groupby('CustomerAddress')['CustomerName'].transform('count') > 1]))


Redundent Name per customer's mails count :  95
Redundent Name per customer's adress count :  11


## CustomerPhone exploration

In [78]:
len(df["CustomerPhone"].unique())

5000

In [66]:
# drop x and whats after it
df['CustomerPhone'] = df['CustomerPhone'].str.split('x').str[0]

In [67]:
# List of phone number patterns
phone_patterns = [
    r'^\d{3}.\d{3}.\d{4}$',                  # 123.456.7890
    r'^(\d{3}-|\(\d{3}\) )?\d{3}-\d{4}$',  # (123) 456-7890 or 123-456-7890
    r'^\+\d{1,2} \(\d{3}\) \d{3}-\d{4}$',  # +1 (123) 456-7890
    r'^\d{3}[-.]?\d{3}[-.]?\d{4}$',        # 123-456-7890 or 1234567890
    r'^1-800-\d{3}-\d{4}$',                # 1-800-123-4567
    r'^(\(\d{3}\) \d{3}-\d{4}|\d{3}[-.]?\d{3}[-.]?\d{4})$',  # (123) 456-7890 or 123-456-7890 or 123.456.7890
]

# Initialize a dictionary to store counts for each pattern
pattern_counts = {pattern: 0 for pattern in phone_patterns}

# Iterate through patterns and count occurrences
for pattern in phone_patterns:
    df['ContainsPattern'] = df['CustomerPhone'].str.contains(pattern)
    count = df['ContainsPattern'].sum()
    pattern_counts[pattern] = count

# Print the counts for each pattern
for pattern, count in pattern_counts.items():
    print(f"Pattern '{pattern}': {count} occurrences")

# Optional: Drop the temporary 'ContainsPattern' column
df.drop(columns=['ContainsPattern'], inplace=True)

Pattern '^\d{3}.\d{3}.\d{4}$': 1983 occurrences
Pattern '^(\d{3}-|\(\d{3}\) )?\d{3}-\d{4}$': 984 occurrences
Pattern '^\+\d{1,2} \(\d{3}\) \d{3}-\d{4}$': 0 occurrences
Pattern '^\d{3}[-.]?\d{3}[-.]?\d{4}$': 2380 occurrences
Pattern '^1-800-\d{3}-\d{4}$': 0 occurrences
Pattern '^(\(\d{3}\) \d{3}-\d{4}|\d{3}[-.]?\d{3}[-.]?\d{4})$': 2380 occurrences


C:\Users\YouCode\AppData\Local\Temp\ipykernel_9848\3907370565.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['ContainsPattern'] = df['CustomerPhone'].str.contains(pattern)


In [57]:
# check for phone regex in CustomerPhone column
print(df['CustomerPhone'].str.contains(r'^\d{3}-\d{3}-\d{4}$').value_counts())

CustomerPhone
False    4624
True      386
Name: count, dtype: int64


## CustomerSegment exploration

In [69]:
df['CustomerSegment'].unique()

array(['Silver', 'Gold', 'Bronze'], dtype=object)

## SupplierName & SupplierLocation & SupplierContact

In [70]:
df["SupplierName"].unique()

array(['Rodriguez, Winters and Perez', 'Lawson-Wilkins', 'Lee-Miller',
       'Soto-Rivera', 'Campos-Murillo', 'Wagner, Jackson and Holt',
       'White and Sons', 'Brown-Miller', 'Colon, Palmer and Welch',
       'Mack, Wright and Mclaughlin', 'Roman-Chambers',
       'Mathis, Olson and Ryan', 'Peck Group', 'Ramirez, Hicks and Lara',
       'Smith PLC', 'Gilbert, Miller and Lee', 'Higgins and Sons',
       'King-Rodriguez', 'Randall-Ballard', 'Thornton, Jackson and Gray'],
      dtype=object)

In [71]:
df['SupplierLocation'].unique()

array(['Irwinhaven', 'Calderonchester', 'South Emilyview', 'West Denise',
       'Lawsontown', 'Port Sarahview', 'West Scott', 'Seanfort',
       'Antonioshire', 'Alexandrachester', 'Dennischester', 'Herreraland',
       'West Lindafurt', 'Dustinmouth', 'Lake Candiceton',
       'Lake Juanland', 'West Johnbury', 'South Lindatown', 'Popeport',
       'Leslieville'], dtype=object)

In [75]:
df.groupby('SupplierName')['SupplierLocation'].transform('count')

0       263
1       244
2       256
3       224
4       263
       ... 
5005    254
5006    254
5007    254
5008    254
5009    254
Name: SupplierLocation, Length: 5010, dtype: int64

In [76]:
df['SupplierContact'].unique()

array(['6538306661', nan, '+1-588-750-7646', '805-650-6257x5876',
       '822-222-4939', '+1-362-752-2777x1817', '288-572-7043',
       '(502)758-2166', '230-757-6575x898', '856-399-9995x715',
       '001-324-909-1637x2831', '(906)306-1777x264', '+1-838-976-6137',
       '(277)930-2705x3206', '459-301-8580', '+1-619-677-8121x28226',
       '230-568-4141', '7724600682', '933.564.7029x6917', '853-738-3918',
       '001-575-242-3572x740'], dtype=object)

## Shipper & ShippingMethod

In [79]:
df['ShipperName'].unique()

array(['and Sons', 'PLC', 'LLC', 'Ltd', 'Inc', 'Group'], dtype=object)

In [80]:
df['ShippingMethod'].unique()

array(['Ground', 'Air', 'Sea'], dtype=object)

## QuantitySold & TotalAmount & DiscountAmount & NetAmount 

In [82]:
df['QuantitySold'].describe()

count    5010.000000
mean       50.827146
std        28.523303
min         1.000000
25%        26.000000
50%        50.000000
75%        76.000000
max       100.000000
Name: QuantitySold, dtype: float64

In [83]:
df['TotalAmount'].describe()

count     5010.000000
mean     28851.529621
std      23139.578225
min         46.230000
25%       8864.130000
50%      23546.135000
75%      42985.950000
max      99513.000000
Name: TotalAmount, dtype: float64

In [84]:
df['DiscountAmount'].describe()

count    5010.000000
mean       86.425236
std        70.574899
min         0.010000
25%        25.685000
50%        69.345000
75%       129.277500
max       298.110000
Name: DiscountAmount, dtype: float64

In [85]:
df['NetAmount'].describe()

count     5010.000000
mean     28765.104385
std      23112.635714
min         33.600000
25%       8774.435000
50%      23424.750000
75%      42886.600000
max      99276.950000
Name: NetAmount, dtype: float64

## StockReceived & Stocksold & StockOnHand

In [86]:
df['StockReceived'].describe()

count    5010.000000
mean      277.074052
std       130.330697
min        50.000000
25%       164.000000
50%       279.000000
75%       393.000000
max       500.000000
Name: StockReceived, dtype: float64

In [87]:
df['StockSold'].describe()

count    5010.000000
mean      139.116966
std       111.016370
min         1.000000
25%        49.000000
50%       109.000000
75%       209.000000
max       498.000000
Name: StockSold, dtype: float64

In [88]:
len(df[df['StockReceived'] < df['StockSold']])

0

In [89]:
df['StockOnHand'].describe()

count    5010.000000
mean      137.957086
std       110.360693
min         0.000000
25%        49.000000
50%       107.000000
75%       206.000000
max       490.000000
Name: StockOnHand, dtype: float64

## Checking relations validation

In [20]:
# Chec fact Sales
df.groupby(["Date", 'ShipperName', 'ShippingMethod',"ProductID", "CustomerName"])['TotalAmount'].count()

Date        ShipperName  ShippingMethod  ProductID                             CustomerName     
01-01-2022  PLC          Air             RobertElectronicsMobile               Joanna Robinson      1
01-01-2023  Inc          Sea             ShawnElectronicsHeadphones            Latoya Reilly        1
01-02-2022  Ltd          Air             KatieBooksNon-fiction                 Charlene Sanchez     1
01-03-2022  and Sons     Air             NonExistentProductToysAction Figures  Christopher Wolfe    1
01-03-2023  LLC          Air             JamieBooksFiction                     Elizabeth Olsen      1
                                                                                                   ..
2023-09-28  PLC          Sea             NathanielElectronicsCamera            Jason Marshall       1
            and Sons     Air             ElizabethClothingShirt                Ashley Carter        1
                         Ground          RobertElectronicsMobile               Kathryn 

In [13]:
grouped = df.groupby(['Date',  'SupplierLocation', 'SupplierName', 'priceJdid'])['StockOnHand'].count()
filtered_grouped = grouped[grouped > 1]

print(len(filtered_grouped))

46


In [14]:
df['priceJdid'] = df['TotalAmount'] / df['QuantitySold']
# round to 0 decimal
df['priceJdid'] = df['priceJdid'].round(0)

In [15]:
grouped = df.groupby(['ProductID'])['priceJdid'].unique()
filtered_grouped = grouped[grouped.str.len() > 1] 

filtered_grouped

ProductID
NonExistentProductBooksAcademic                                     [952.0, 693.0]
NonExistentProductBooksFiction                                      [433.0, 930.0]
NonExistentProductClothingShirt                                     [995.0, 840.0]
NonExistentProductElectronicsHeadphones        [660.0, 873.0, 651.0, 461.0, 229.0]
NonExistentProductInvalidCategoryAcademic                           [693.0, 952.0]
NonExistentProductInvalidCategoryFiction                            [433.0, 930.0]
NonExistentProductInvalidCategoryHeadphones           [651.0, 461.0, 873.0, 660.0]
NonExistentProductInvalidCategoryShirt                              [840.0, 995.0]
Name: priceJdid, dtype: object